Kotug Project, Group 1

Allan Guzman, June Chen, Sarah Blanc

Let's maybe first remember here the research questions of this project:

**Main Research Question**:

"How can the reliability and efficiency of Kotug OptiPort's tugboat scheduling tool be enhanced through the development and evaluation of novel methods for predicting pickup and drop-off coordinates and other relevant variables?"

**Subresearch Questions**:

- "What advanced predictive methods can be developed and implemented to accurately forecast the coordinates (latitude and longitude) for pickup and drop-off locations of tugs on vessels in the port of Rotterdam?"
- "Which variables, including AIS data, customer requirements, and port-specific constraints (or lack thereof), play a significant role in determining the pickup and drop-off locations and timing of tugs, as well as the required number of tugs for each vessel operation in the port of Rotterdam, and why are they important?"
- "Which of the above mentioned variables can be combined in order to increase the reliability of the results, and why can they be combined?"

The work on this first research question in done in this file, while the 2 other subquestions are answered in the file called "Data_preparation". Finally, this file also contains the conclusion of this project (see end of the file).


**Predictions**

In this notebook, the primary objective is to predict the pick-up and drop-off locations of vessels. Given the variation in data availability across different harbors, a strategic approach was adopted. We specifically target a single harbor for analysis. The flexibility of our model allows us to select any harbor for analysis simply by modifying a designated variable in the code.

To elaborate, we begin by choosing a harbor with substantial data volume. All relevant data pertaining to incoming and departing vessels from this selected harbor is extracted. Our model is then applied to this subset of data. Notably, the ability to change the harbor variable in the code grants the flexibility to extend this analysis to other harbors seamlessly.

Additionally, it is crucial to handle incoming and departing vessels separately in our analysis. Within the chosen harbor's dataset, we further categorize the data. Leaving vessels form one subset, while incoming vessels constitute another. This approach is grounded in the inherent knowledge possessed by vessels and drivers. Specifically, vessels are inherently aware of their status as incoming or departing, and they are assigned to specific ports. These foundational assumptions underpin our analytical framework, ensuring a focused and accurate prediction methodology.

Literature review that explains the path followed through the different methods used:

As part of our effort to answer the first research question and to identify appropriate techniques for geospatial data processing, we painstakingly carried out a literature research. There were two goals in mind: finding viable approaches and understanding how they may be used in the field. Our investigation produced important findings that shaped our methodology. The key findings were the following:
1. Integrated prediction over isolated coordinates: We could have been tempted at first to use several regression models to predict latitude and longitude independently. The literature, however, strongly advises against this practise. Researchers and academics often advise against this segmentation. This advice is supported by the fact that latitude and longitude have complex interactions that are difficult to fully comprehend when viewed as separate concepts. As a result, we began to concentrate on holistic approaches that take into account all of these factors at once.
2. The potential of decision tree classifiers on our discretized polygon of water: Among the methods for classifying our data that showed promise was the decision tree classifier, especially when it came to pick-up and drop-off location prediction. It is not without restrictions, despite its potential. Indeed, we could use it, however we should create a dependence between squares (notable squares that are close to each other as an example). This could be done via new variables that we do not have. Developping them would be too complicated to apply in this project. Furthermore, its failure to adequately incorporate the geospatial nuances of our scenario is a major limitation. This insight drove us to read more in the literature and look for techniques that include the geographical dimension from the beginning. However, you can still see the results from this first method below.  
3.  Introducing kriging, a spatial regression method: Our research introduced us to the Kriging technique, which is a very relevant method for our needs regarding the classification of geospatial data. This could be done on our discretized polygon of water. Furthermore, it is a reliable framework for position prediction that takes into consideration the underlying spatial patterns. Using the spatial correlations found in our data, this approach solves our categorization problem in a way that is contextually rich and nuanced. Unfortunately, after reading a lot about it, we again remarked that this method asks for a lot of complicated work before applying the method. We do not have the knowledges now to handle it in a way that allows us to produce convincing results. Thus, we decided to search for further possible methods.
4. We reached the consensus that it's essential to reconsider approaches that don't rely on our discretized polygon. This broader perspective ensures we're not confined by our initial decision to go for this polygon. Following a brainstorming session and further reading of papers, we determined that ...... TO BE COMPLETED


Import of the librairies

In [78]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import matplotlib.pyplot as plt
import re
import seaborn as sn

import folium
import rasterio
import rasterio.mask

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from geopy.distance import geodesic
import folium

from shapely.geometry import Point, Polygon, LineString
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from pyproj import Geod

Import of the data

In [83]:
incoming_weather_size_draught = gpd.read_file('Data/Input_ML/incoming_weather_size_draught.geojson')
incoming_weather_size = gpd.read_file('Data/Input_ML/incoming_weather_size.geojson')
incoming_weather = gpd.read_file('Data/Input_ML/incoming_weather.geojson')

In [84]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
incoming_weather_size_draught['Wind_Direction'] = label_encoder.fit_transform(incoming_weather_size_draught['Wind_Direction_Cat2'])
incoming_weather_size_draught['Type_Cat'] = label_encoder.fit_transform(incoming_weather_size_draught['Type'])
incoming_weather_size_draught['Type'] = label_encoder.fit_transform(incoming_weather_size_draught['Type'])
#incoming_weather_size_draught['Wind_speed'] = label_encoder.fit_transform(incoming_weather_size_draught['Wind_speed'])
#incoming_weather_size_draught['to_haven'] = label_encoder.fit_transform(incoming_weather_size_draught['to_haven'])
incoming_weather_size_draught

,area_ID,center,from,from_rounded,to_haven,trip_ID,vessel_mmsi,area,Wind_Direction_Cat1,Wind_Direction_Cat2,Wind_Speed,d_from_haven,d_to_haven,Lenght,Width,Type,Navigation_draught,geometry,Wind_Direction,Type_Cat
0,1584,POINT (4.14418460267196 51.96555611267634),2022-05-31 23:34:58+00:00,2022-06-01 00:00:00+00:00,3PET,4,538005865,"POLYGON ((4.141337398344955 51.96305993744309,...",SE,S,30,5696.698651,0.000000,159.0,27.0,2,8.3,POINT (4.14163 51.96557),2,2
1,2335,POINT (4.2649075344304945 51.90749906115679),2022-11-24 02:30:45+00:00,2022-11-24 03:00:00+00:00,3PET,11529,538005865,"POLYGON ((4.268520679547496 51.90582746090193,...",SW,S,100,5067.267729,629.430921,159.0,27.0,2,7.6,POINT (4.26256 51.90621),2,2
2,1186,POINT (4.0808868772623645 51.97886976519478),2022-06-01 01:11:15+00:00,2022-06-01 01:00:00+00:00,PET8,7,538007563,"POLYGON ((4.084104921803812 51.97577826556335,...",SW,S,30,3070.369645,2905.398848,276.0,48.0,2,13.7,POINT (4.08005 51.97787),2,2
3,1067,POINT (4.061847847593366 51.98531701165355),2022-06-25 22:26:59+00:00,2022-06-25 22:00:00+00:00,PET8,1588,538007563,"POLYGON ((4.058668265563304 51.98849659368361,...",S,S,20,4910.943305,1064.825188,276.0,48.0,2,13.5,POINT (4.05872 51.98483),2,2
4,1186,POINT (4.0808868772623645 51.97886976519478),2022-06-01 01:20:30+00:00,2022-06-01 01:00:00+00:00,AMAZ,8,218366000,"POLYGON ((4.084104921803812 51.97577826556335,...",SW,S,30,4961.796641,1897.204224,335.0,42.0,0,13.2,POINT (4.07866 51.97826),2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2542,1067,POINT (4.061847847593366 51.98531701165355),2022-12-24 16:29:47+00:00,2022-12-24 16:00:00+00:00,AMAZ,13621,538005844,"POLYGON ((4.058668265563304 51.98849659368361,...",SW,S,50,6255.953298,603.047567,368.0,51.0,0,11.3,POINT (4.06404 51.98374),2,0
2543,1622,POINT (4.1507481542195 51.953428387499024),2022-12-24 20:17:36+00:00,2022-12-24 20:00:00+00:00,CKVTTI,13626,538006893,MULTIPOLYGON (((4.147696562405082 51.950341609...,S,S,60,3770.244499,4423.591224,160.0,25.0,2,6.9,POINT (4.15134 51.95288),2,2
2544,2414,POINT (4.277982400813358 51.90190160065854),2022-12-24 23:10:17+00:00,2022-12-24 23:00:00+00:00,3PET,13632,353456000,"POLYGON ((4.28123900766775 51.8994682968418, 4...",S,S,70,3823.124167,1873.574484,148.0,23.0,2,8.0,POINT (4.27695 51.90052),2,2
2545,1661,POINT (4.158229272691092 51.94804575336068),2022-12-25 00:56:07+00:00,2022-12-25 01:00:00+00:00,7PET,13637,538003962,POLYGON ((4.154055726465209 51.950341609322834...,S,S,60,5012.121838,4181.605113,183.0,32.0,2,11.3,POINT (4.15802 51.95009),2,2


In [85]:
leaving_weather_size_draught = gpd.read_file('Data/Input_ML/leaving_weather_size_draught.geojson')
leaving_weather_size = gpd.read_file('Data/Input_ML/leaving_weather_size.geojson')
leaving_weather = gpd.read_file('Data/Input_ML/leaving_weather.geojson')

In [86]:
leaving_weather_size_draught


,area_ID,center,to,to_rounded,from_haven,trip_ID,vessel_mmsi,area,Wind_Direction_Cat1,Wind_Direction_Cat2,Wind_Speed,d_from_haven,d_to_haven,Lenght,Width,Type,Navigation_draught,geometry
0,1224,POINT (4.087284503833875 51.966239519473156),2022-06-01T00:30:17+00:00,2022-06-01 01:00:00+00:00,AMAZ,1,355271000,"POLYGON ((4.084104921803812 51.96941910150322,...",SW,S,30,3685.167505,3173.833360,336.0,46.0,cargo,11.3,POINT (4.08905 51.96774)
1,1224,POINT (4.087284503833875 51.966239519473156),2022-12-06T19:47:53+00:00,2022-12-06 20:00:00+00:00,AMAZ,12473,355271000,"POLYGON ((4.084104921803812 51.96941910150322,...",N,N,70,3736.296079,3122.704786,336.0,46.0,cargo,12.2,POINT (4.08844 51.96875)
2,2893,POINT (4.3541802773739935 51.89784648701013),2022-06-01T01:46:24+00:00,2022-06-01 02:00:00+00:00,WAAL,6,564002000,"POLYGON ((4.351189812329148 51.8994682968418, ...",W,W,70,7008.924189,693.166016,174.0,27.0,tanker,6.9,POINT (4.35410 51.89852)
3,1224,POINT (4.087284503833875 51.966239519473156),2022-06-01T01:56:33+00:00,2022-06-01 02:00:00+00:00,AMAZ,5,636092784,"POLYGON ((4.084104921803812 51.96941910150322,...",W,W,70,3759.030003,3099.970862,300.0,46.0,cargo,10.9,POINT (4.08988 51.96918)
4,1186,POINT (4.0808868772623645 51.97886976519478),2022-07-08T13:28:19+00:00,2022-07-08 13:00:00+00:00,AMAZ,2398,636092784,"POLYGON ((4.084104921803812 51.97577826556335,...",W,W,70,4585.359011,2273.641854,300.0,46.0,cargo,11.0,POINT (4.08271 51.97634)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,2974,POINT (4.3664131538129665 51.901526425867466),2022-12-24T22:44:59+00:00,2022-12-24 23:00:00+00:00,WAAL,13629,538004170,"POLYGON ((4.370267304509529 51.8994682968418, ...",S,S,70,6262.311218,1439.778988,228.0,32.0,tanker,8.7,POINT (4.36411 51.90007)
2298,1424,POINT (4.11923161276585 51.96652963319716),2022-12-25T02:58:17+00:00,2022-12-25 03:00:00+00:00,CKVTTI,13638,636015170,"POLYGON ((4.115900742104447 51.96941910150322,...",S,S,60,6395.202161,1798.633563,230.0,38.0,cargo,7.8,POINT (4.12067 51.96743)
2299,1820,POINT (4.182629270525371 51.94040644412748),2022-12-25T07:31:59+00:00,2022-12-25 08:00:00+00:00,CKVTTI,13644,538006893,MULTIPOLYGON (((4.185851546765845 51.937623281...,S,S,50,1061.080165,7132.755559,160.0,25.0,tanker,6.9,POINT (4.18287 51.93777)
2300,2493,POINT (4.290930864333445 51.897467449818606),2022-12-25T10:35:05+00:00,2022-12-25 11:00:00+00:00,3PET,13653,538009704,"POLYGON ((4.287598171727877 51.8994682968418, ...",S,S,50,2470.305116,3226.393535,140.0,21.0,tanker,6.5,POINT (4.29352 51.89772)


Encoding of the data that are not numerical

In [87]:
# Assuming 'column_name' is the name of the column containing strings in your DataFrame
label_encoder = LabelEncoder()
leaving_weather_size_draught['Wind_Direction'] = label_encoder.fit_transform(leaving_weather_size_draught['Wind_Direction_Cat2'])
leaving_weather_size_draught['Wind_Speed'] = label_encoder.fit_transform(leaving_weather_size_draught['Wind_Speed'])
leaving_weather_size_draught['Type'] = label_encoder.fit_transform(leaving_weather_size_draught['Type'])
#leaving_weather_size_draught['from_haven'] = label_encoder.fit_transform(leaving_weather_size_draught['from_haven'])


Code to know what are the 2 most used harbour for the incoming and leaving vessels

In [88]:
leaving_top = leaving_weather_size_draught['from_haven'].value_counts()
incoming_top = incoming_weather_size_draught['to_haven'].value_counts()
print(leaving_top[0:2])
print(incoming_top[0:2])

from_haven
EURO      451
AMALIA    309
Name: count, dtype: int64
to_haven
EURO    480
7PET    377
Name: count, dtype: int64


Extraction of the data that concerns the most used harbours (one for leaving vessels and one for incoming vessels)

In [90]:
incoming_harbor = 'EURO'
leaving_harbor = 'EURO'

leaving_weather_size_draught_top = leaving_weather_size_draught[leaving_weather_size_draught['from_haven']==leaving_harbor]
incoming_weather_size_draught_top = incoming_weather_size_draught[incoming_weather_size_draught['to_haven']==incoming_harbor]


In [96]:
columns_leaving_trajectories = [
    'from_haven',
    'Wind_Direction',
    'Wind_Speed',
    'Lenght',
    'Width',
    'Type',
    'd_from_haven'
]

columns_incoming_trajectories = [
    'to_haven',
    'Wind_Direction',
    'Wind_Speed',
    'Lenght'	,
    'Width'	,
    'Type',
    'd_to_haven'
]

columns_leaving_areas = [
    #'from_haven',
    'Wind_Direction',
    'Wind_Speed',
    'Lenght',
    'Width',
    'Type',
]

columns_incoming_areas = [
    #'to_haven',
    'Wind_Direction',
    'Wind_Speed',
    'Lenght'	,
    'Width'	,
    'Type',
]


Check of potential correlations in the data

In [31]:
# corr_matrix_incoming = incoming_weather_size_draught[columns_incoming].corr()
# sn.heatmap(corr_matrix_incoming, annot=True)
# plt.show()

In [32]:
# corr_matrix_leaving = leaving_weather_size_draught[columns_leaving].corr()
# sn.heatmap(corr_matrix_leaving, annot=True)
# plt.show()

As observed, aside from Length and Width, there are no notably high correlations within the datasets.

**BELOW, YOU CAN SEE THE IMPLEMENTATION OF THE POINT 2 FROM THE LITERATURE REVIEW: THE DECISION TREE ON THE DISCRETIZED POLYGON.**

# Trayectories approach

In [135]:
incoming_trajectories = incoming_weather_size_draught[columns_incoming_trajectories]
incoming_trajectories = incoming_trajectories.groupby('to_haven').apply(lambda x: x).drop(columns = 'to_haven')


KeyError: "['Wind_Direction'] not in index"

In [131]:
X = incoming_trajectories.drop(columns= 'd_to_haven')
y = incoming_trajectories['d_to_haven']

amalia = incoming_trajectories.loc['EURO']


In [132]:
amalia

,Wind_Direction,Wind_Speed,Lenght,Width,Type,d_to_haven
11,3,40,147.0,24.0,2,5016.732361
13,1,80,400.0,59.0,0,810.734272
15,0,20,400.0,59.0,0,358.472333
16,3,60,300.0,43.0,0,345.730126
17,3,100,300.0,43.0,0,2060.942407
...,...,...,...,...,...,...
2516,2,60,300.0,48.0,0,1693.118113
2532,0,50,176.0,31.0,2,2237.200050
2533,3,110,261.0,33.0,0,928.901407
2540,3,70,88.0,14.0,0,0.000000


In [133]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load your DataFrame (assuming it's in a variable 'df')
# Example code to load data from a CSV file:
# df = pd.read_csv('your_data.csv')

# Define your features and target variable
# In this case, 'd_to_haven' is the target variable, and the other columns are features.
X = amalia.drop('d_to_haven', axis=1)  # Features
y = amalia['d_to_haven']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Add the predicted distances to the original DataFrame
X_test['predicted_distance'] = y_pred

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared (R2): {r2:.2f}")

# You can also access feature importances
feature_importances = rf_model.feature_importances_
for feature, importance in zip(X.columns, feature_importances):
    print(f"{feature}: {importance:.2f}")

# Now, your DataFrame 'df' contains a new column 'predicted_distance' with the predictions.


Mean Squared Error: 1632309.64
R-squared (R2): -0.23
Wind_Direction: 0.12
Wind_Speed: 0.19
Lenght: 0.49
Width: 0.18
Type: 0.02


In [134]:
X_test['original_distance'] = amalia['d_to_haven'].loc[list(X_test.index.values)]
X_test['diff_dist'] = abs(X_test['predicted_distance']  - X_test['original_distance'])
print(X_test['diff_dist'].mean())

889.835638583762


In [116]:
def trajectory_classification(grouped_df, haven):
    haven_data = grouped_df.loc[haven]
    X = haven_data.drop(columns= 'd_to_haven')
    y = haven_data['d_to_haven']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
    


to_haven      
3PET      0          0.000000
          1        629.430921
          23      1795.935183
          24      1303.474140
          29      2347.385090
                     ...     
YANG      2473    1154.566618
          2483    1778.543152
          2485       0.000000
          2488     974.118683
          2497    8628.836551
Name: d_to_haven, Length: 2547, dtype: float64

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

# Areas approach

**Prediction for LEAVING**

In [97]:
X = leaving_weather_size_draught_top[columns_leaving_areas]
y = leaving_weather_size_draught_top['area_ID']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=3)
#X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(leaving_weather_size_draught[columns_leaving], leaving_weather_size_draught['area_ID'], test_size=0.3, random_state=42)
data = X_test

In [98]:
print("Number of features in training data:", X_train.shape[1]) 
print("Number of features in test data:", X_test.shape[1])

Number of features in training data: 5
Number of features in test data: 5


Decision Tree Classifier

In [99]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.datasets import load_iris


# Initialize the Decision Tree Classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=3)

# Train the classifier using the training data
x = decision_tree_classifier.fit(X_train, y_train)

# Make predictions on the test data
predictions = decision_tree_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)

print("Accuracy of the Decision Tree Classifier: {:.2f}%".format(accuracy * 100))

# Add predictions to the original DataFrame

data['Predicted_Area_ID'] = predictions




Accuracy of the Decision Tree Classifier: 19.03%


In [100]:
merged_df = pd.merge(data, leaving_weather_size_draught, left_index=True, right_index=True, how='left')
#merged_df

In [101]:
# Create a new empty list to store prediction_center values
prediction_centers = []

# Iterate through the rows of merged_df
for index, row in merged_df.iterrows():
    # Get the Predicted_Area_ID value from the current row of merged_df
    predicted_area_id = row['Predicted_Area_ID']
    
    # Find the corresponding row in incoming_weather_size_draught where area_ID matches Predicted_Area_ID
    matching_row = leaving_weather_size_draught[leaving_weather_size_draught['area_ID'] == predicted_area_id]
    
    # Check if a matching row is found
    if not matching_row.empty:
        # Get the value from the 'center' column in the matching row
        prediction_center = matching_row['center'].values[0]
    else:
        # If no matching row is found, assign a default value (you can change this as per your requirement)
        prediction_center = None
    
    # Append the prediction_center value to the list
    prediction_centers.append(prediction_center)

# Add the prediction_centers list as a new column to merged_df
merged_df['prediction_center'] = prediction_centers

#merged_df

In [103]:
merged_df

,Wind_Direction_x,Wind_Speed_x,Lenght_x,Width_x,Type_x,Predicted_Area_ID,area_ID,center,to,to_rounded,...,Wind_Speed_y,d_from_haven,d_to_haven,Lenght_y,Width_y,Type_y,Navigation_draught,geometry,Wind_Direction_y,prediction_center
849,1,6,399.0,54.0,0,1225,1224,POINT (4.087284503833875 51.966239519473156),2022-07-14T22:26:04+00:00,2022-07-14 22:00:00+00:00,...,6,3387.739178,2995.231210,399.0,54.0,0,10.0,POINT (4.08806 51.96917),1,POINT (4.087284503833875 51.972598683533285)
375,2,5,275.0,40.0,0,1224,1225,POINT (4.087284503833875 51.972598683533285),2022-07-02T07:40:01+00:00,2022-07-02 08:00:00+00:00,...,5,3576.123183,2806.847206,275.0,40.0,0,11.7,POINT (4.08813 51.97141),2,POINT (4.087284503833875 51.966239519473156)
954,1,5,306.0,40.0,0,1183,1186,POINT (4.0808868772623645 51.97886976519478),2022-07-07T22:37:41+00:00,2022-07-07 23:00:00+00:00,...,5,4275.508552,2107.461837,306.0,40.0,0,13.3,POINT (4.08239 51.97785),1,POINT (4.0809253397737475 51.95988035541303)
691,1,1,220.0,33.0,0,1225,1063,POINT (4.061780049795423 51.96113970048921),2022-07-18T11:17:17+00:00,2022-07-18 11:00:00+00:00,...,1,841.050069,5541.920319,220.0,33.0,0,8.2,POINT (4.06466 51.96093),1,POINT (4.087284503833875 51.972598683533285)
1948,2,4,336.0,45.0,0,1067,1225,POINT (4.087284503833875 51.972598683533285),2022-10-21T00:29:28+00:00,2022-10-21 00:00:00+00:00,...,4,3693.376653,2689.593735,336.0,45.0,0,12.6,POINT (4.08997 51.97331),2,POINT (4.061847847593366 51.98531701165355)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,2,6,220.0,33.0,0,1185,1183,POINT (4.0809253397737475 51.95988035541303),2022-12-22T12:28:17+00:00,2022-12-22 12:00:00+00:00,...,6,2046.496530,4336.473858,220.0,33.0,0,9.2,POINT (4.07947 51.96167),2,POINT (4.080970466897103 51.972806813886265)
1619,0,6,328.5,46.0,0,1183,1225,POINT (4.087284503833875 51.972598683533285),2022-09-02T14:12:05+00:00,2022-09-02 14:00:00+00:00,...,6,3723.758056,2659.212332,328.5,46.0,0,11.4,POINT (4.08710 51.97277),0,POINT (4.0809253397737475 51.95988035541303)
184,3,4,294.0,32.0,0,1224,988,POINT (4.0491335556626495 51.99162927499277),2022-07-05T09:19:56+00:00,2022-07-05 09:00:00+00:00,...,4,6382.970388,0.000000,294.0,32.0,0,12.3,POINT (4.04934 51.99057),3,POINT (4.087284503833875 51.966239519473156)
1476,2,4,194.0,28.0,0,1063,1224,POINT (4.087284503833875 51.966239519473156),2022-08-16T10:02:51+00:00,2022-08-16 10:00:00+00:00,...,4,3180.699821,3202.270568,194.0,28.0,0,9.8,POINT (4.08631 51.96709),2,POINT (4.061780049795423 51.96113970048921)


In [102]:
# Function to convert string representation of coordinates to actual coordinates
def parse_coordinates(coord_str):
    if coord_str:
        # Extract numerical values from the string (format: 'POINT (longitude latitude)')
        coordinates = re.findall(r"[-+]?\d*\.\d+|\d+", coord_str)
        lat, lon = map(float, coordinates)
        return lat, lon
    else:
        return None

# Convert 'from_location' and 'center' columns to actual coordinates
merged_df['to_location_coords'] = merged_df['to_location'].apply(parse_coordinates)
merged_df['center_coords'] = merged_df['prediction_center'].apply(parse_coordinates)

# Function to calculate distance between two points using geodesic
def calculate_distance(row):
    if row['to_location_coords'] and row['center_coords']:
        return geodesic(row['to_location_coords'], row['center_coords']).meters
    else:
        return None

# Apply the calculate_distance function to the DataFrame to compute distances
merged_df['distance_to_predicted_center'] = merged_df.apply(calculate_distance, axis=1)
moyenne_distance = merged_df['distance_to_predicted_center'].mean()

# Afficher la moyenne
print("Average distance between predicted and real locations: ", moyenne_distance, "meters")



KeyError: 'to_location'

**Predictions for INCOMING**

In [18]:
X = incoming_weather_size_draught_top[columns_incoming_areas]
y = incoming_weather_size_draught_top['area_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=3)
#X_from_lat_train, X_from_lat_test, y_from_lat_train, y_from_lat_test = train_test_split(leaving_weather_size_draught[columns_leaving], leaving_weather_size_draught['area_ID'], test_size=0.3, random_state=42)
data = X_test

In [19]:
print("Number of features in training data:", X_train.shape[1])
print("Number of features in test data:", X_test.shape[1])

Number of features in training data: 5
Number of features in test data: 5


In [20]:

# Initialize the Decision Tree Classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=3)

# Train the classifier using the training data
x = decision_tree_classifier.fit(X_train, y_train)

# Make predictions on the test data
predictions = decision_tree_classifier.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)

print("Accuracy of the Decision Tree Classifier: {:.2f}%".format(accuracy * 100))

# Add predictions to the original DataFrame

data['Predicted_Area_ID'] = predictions


Accuracy of the Decision Tree Classifier: 12.92%


,Direction,Wind_speed,Lenght,Width,Type,Predicted_Area_ID
219,2,1,299.0,49.0,0,1224
772,5,3,300.0,40.0,0,1027
600,8,0,243.0,32.0,0,1103
2258,7,1,366.0,48.0,0,1067
2033,5,0,333.0,48.0,0,1107
...,...,...,...,...,...,...
407,7,1,270.0,40.0,0,1106
1780,5,3,300.0,40.0,0,1027
1224,5,3,300.0,43.0,0,1106
3032,6,1,300.0,48.0,0,1186


In [21]:
merged_df = pd.merge(data, incoming_weather_size_draught, left_index=True, right_index=True, how='left')
#merged_df

In [22]:
# Create a new empty list to store prediction_center values
prediction_centers = []

# Iterate through the rows of merged_df
for index, row in merged_df.iterrows():
    # Get the Predicted_Area_ID value from the current row of merged_df
    predicted_area_id = row['Predicted_Area_ID']
    
    # Find the corresponding row in incoming_weather_size_draught where area_ID matches Predicted_Area_ID
    matching_row = incoming_weather_size_draught[incoming_weather_size_draught['area_ID'] == predicted_area_id]
    
    # Check if a matching row is found
    if not matching_row.empty:
        # Get the value from the 'center' column in the matching row
        prediction_center = matching_row['center'].values[0]
    else:
        # If no matching row is found, assign a default value (you can change this as per your requirement)
        prediction_center = None
    
    # Append the prediction_center value to the list
    prediction_centers.append(prediction_center)

# Add the prediction_centers list as a new column to merged_df
merged_df['prediction_center'] = prediction_centers

#merged_df

In [24]:
# Function to convert string representation of coordinates to actual coordinates
def parse_coordinates(coord_str):
    if coord_str:
        # Extract numerical values from the string (format: 'POINT (longitude latitude)')
        coordinates = re.findall(r"[-+]?\d*\.\d+|\d+", coord_str)
        lat, lon = map(float, coordinates)
        return lat, lon
    else:
        return None

# Convert 'to_location' and 'center' columns to actual coordinates
merged_df['from_location_coords'] = merged_df['from_location'].apply(parse_coordinates)
merged_df['center_coords'] = merged_df['prediction_center'].apply(parse_coordinates)

# Function to calculate distance between two points using geodesic
def calculate_distance(row):
    if row['from_location_coords'] and row['center_coords']:
        return geodesic(row['from_location_coords'], row['center_coords']).meters
    else:
        return None

# Apply the calculate_distance function to the DataFrame to compute distances
merged_df['distance_to_predicted_center'] = merged_df.apply(calculate_distance, axis=1)
moyenne_distance = merged_df['distance_to_predicted_center'].mean()

# Afficher la moyenne
print("Average distance between predicted and real locations: ", moyenne_distance, "meters")


Average distance between predicted and real locations:  1418.1801243934678 meters


**BELOW, YOU CAN SEE THE IMPLEMENTATION OF THE POINT 4 FROM THE LITERATURE REVIEW: ... TO BE COMPLETED.**

**CONCLUSION**

...